![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Travailler avec des données ouvertes Partie 4: 

## Atterrissages de météorites et chutes partie 1

Dans cette troisième partie de notre série de didacticiels sur les données ouvertes, nous utiliserons un ensemble de données de l'année et l'emplacement de toutes les chutes de météorites enregistrées jusqu'en 2013. Ces données sont hébergées sur [ce dépôt github] (https: // github. com / fleiser / Meteorite-landings / blob / master / Meteorite_Landings.csv).


Comme il s'agit de la troisième partie de la série de didacticiels, continuons avec notre première étape traditionnelle et importons les bibliothèques requises. Notez que cette fois, nous utilisons un peu plus de bibliothèques que par le passé. Ne vous inquiétez pas, ce sont des outils supplémentaires pour rendre certaines visualisations un peu plus intéressantes. Nous avons saisi des commentaires décrivant la fonction de chaque bibliothèque et nous vous expliquerons comment nous les utiliserons lorsqu'ils apparaîtront dans le didacticiel.

In [ ]:
# These first three libraries are nothing new!
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Geopandas is very similar to pandas, however it contains extra functionality to work with 
# geospacial data such as latitude and longitude
import geopandas as gpd
# This imports the "Point" function, or a function that creates a point on a geospacial graph.
# this function simply makes things easier to work with in Pandas
from shapely.geometry import Point

# The following three libraries are included in order to make interactive widgets in this notebook.
# More on this later. 
from IPython import display
from ipywidgets import interact_manual
import ipywidgets as widgets



Ne vous inquiétez pas trop de ces nouvelles bibliothèques. Ils seront considérablement moins intimidants une fois que vous verrez à quel point ils ressemblent à ce que nous avons déjà utilisé!

## Rassembler les données

Nous sommes maintenant prêts à télécharger les données du site Github dans la cellule ci-dessous. Vous remarquerez que nous l'avons divisé en quelques étapes supplémentaires dans la cellule ci-dessous. Celles-ci visent à simplifier la gestion des données. Leur fonctionnalité est commentée dans la cellule ci-dessous pour votre commodité.

In [ ]:
'''
These first two lines should not be surprising anymore! This is how we've been bringing data into
our Jupyter notebooks throughout this tutorial series. 
'''
url = 'https://github.com/fleiser/Meteorite-landings/raw/master/Meteorite_Landings.csv'
landings = pd.read_csv(url)

'''
Here things are new and exciting. What the `pd.to_datetime' function does is convert some text that looks 
like a date into a 'datetime' object inside of pandas. This is convenient for parsing later as we will be able
to search by year, month, and day. We're also using errors = 'coerce', which tells python to ignore any data
points that cannot be coerced into a datetime object. You'll also note that we're redefining our pandas column
'year' in place
'''
landings['year'] = pd.to_datetime(landings['year'], errors = 'coerce')
print("Number of observed meteorites in data set:", len(landings))
landings.head()

Vous remarquerez peut-être que la colonne «year» contient également des mois et des jours dans notre dataframe. Vous remarquerez également que la date ici est toujours le premier janvier. Ce n'est pas le résultat d'une grande coïncidence cosmique. Le mois et le jour restent des artefacts du jeu de données, et le mois et le jour de ces données n'existent pas. En tant que tel, nous ignorerons ces quantités dans notre analyse ultérieure. Le mois et la date ne sont pas inclus de manière malveillante - il s'agit plus que probablement d'un artefact de conversions de date et heure de l'ensemble de données d'origine. Avec l'apparition du mois et des jours, ces données peuvent être involontairement trompeuses.

# Créer une carte

En utilisant la latitude et la longitude de chaque météore, nous pouvons créer une carte qui nous permettra d'observer les endroits où les météorites sont tombées ou ont été découvertes. Pour ce faire, nous avons besoin de «géopandas» ou «pandas géo-spatiaux». Geopandas se comporte de manière identique aux pandas, mais contient des fonctionnalités supplémentaires qui facilitent la création de cartes. Notre première tâche consiste à créer un bloc de données géopandas, qui se fait dans la cellule de code ci-dessous.

In [ ]:
# Here we're creating a function which defines our points that we will plot on a map
def create_point(row):
    '''
    Here 'row' is the row of a dataframe that we will use the apply() with this 
    function on. As well, reclong and reclat are the latitude and longitude from our landings 
    data frame. Finally, Point is the Point function that we imported in the beginning of the notebook
    So, all this functio does is return a Point object of latitude and longitude. Easy!
    
    '''
    return Point(row.reclong, row.reclat)

# Now, we create our points by using the apply function on our landings data frame 
points = landings.apply(create_point, axis=1)

print("Below is what our create_point function creates")
print()
print(points.head())

# We are now creating a geopandas data frame with a 'geometry' column as defined 
# by the points we just created. Besides that it is identical to our 'landings'
# data frame 

geo_landings = gpd.GeoDataFrame(landings, geometry=points)

# We also need to define the map projection we're using. In this case, epsg:4326 is the most 
# common projection for a rectangular map. 

geo_landings.crs = {'init': 'epsg:4326'}

# View the first rows, note our 'geometry' tab 
geo_landings.head()

Maintenant que nous avons créé un cadre de données géopandas, voyons comment le représenter sur une carte ! Tout d'abord, créons une carte du monde simple, comme ci-dessous.

In [ ]:
# This creates a world map for us! This data set is a part of geopands and included
# in the hub. 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Now we're just creating a plot as we've done using regular pandas data frames

ax = world.plot(figsize=(20,10), linewidth=0.25, edgecolor='white', color='lightgrey')

Maintenant que nous avons une carte de base, mettons nos débarquements de météorites dessus

In [ ]:

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')

# Because we have a 'geometry' column, geopandas will plot our meteorite landings without 
# us needing to specify which column to plot. However, we do need to specify that it should appear on the same 
# axis as 
geo_landings.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
ax.axis('off')
plt.show()


Bien sûr, il y a environ 45 000 météorites observées sur la parcelle ci-dessus. Ce qui devient désordonné assez rapidement. Il pourrait être plus intéressant de voir les chutes et les trouvailles de météorites pour chaque année. La manière la plus simple de le faire dans un bloc-notes Jupyter consiste à utiliser des widgets interactifs, certaines des bibliothèques importées précédemment. Avant de commencer à créer le widget, parcourons d'abord comment créer une carte de toute année d'intérêt. Le premier ordre du jour: comment extraire l’année d’une colonne date / heure?

In [ ]:
'''
To extract the year from a datetime object, we have to be aware of two methods:

dt   : This stands for "datetime" and is used to cast the text in our data frame into a datetime object

year : This extracts the year from the date time object

'''
# This is how we get just the year out!
print("Syntax 1 output:")
print( geo_landings["year"].dt.year.head() )
print()

# Note that this is equivalent, however the first ".year" is in reference to the column name
# and not a method of the date time object.
print("Syntax 2 output:")
print( geo_landings.year.dt.year.head() )

# If our 'year' column was called 'time', this would look like
# geo_landings.time.dt.year.head()


In [ ]:

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')

# Notice how we're specifying only a single year in this line! 
plot_single_year = geo_landings[geo_landings.year.dt.year == 2012]

plot_single_year.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
plt.title("Meteor Falls in 2012", size = 20)
ax.axis('off')
plt.show()

N'hésitez pas à changer le code ci-dessus pour regarder n'importe quelle année afin de vous familiariser avec la syntaxe. Vous avez peut-être remarqué qu'il y a un point particulier au large de la côte africaine dans l'océan Atlantique. Regardons de plus près ce qui se passe dans la cellule ci-dessous.

In [ ]:
# Here we're looking at the last 10 entries of our dataframe
plot_single_year.tail(10)

Ah ha! Il y a le problème. Dans ce cas, les météorites "Afrique du Nord-Ouest" n'ont pas de coordonnées valides et sont simplement remplies de zéro. Ceci est un autre cas de "données médiocres" qui ne sont pas intentionnellement trompeuses, mais peuvent conduire à des résultats étranges si vous ne faites pas attention. L'une des manières les plus courantes de traiter les données manquantes dans de nombreux cas consiste simplement à remplacer ces valeurs manquantes par zéro, il est donc préférable de faire attention. Dans ce cas, filtrons ces points pour qu'ils n'apparaissent pas sur notre carte.

In [ ]:

# If you want to have multiple conditions, the syntax is the same, but each condition is separated
# by an ampersand (&) for 'and'. Also note that '!=' means "not equal to" 

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')
# Note the changes in the line below
plot_single_year = geo_landings[(geo_landings.year.dt.year == 2012) & (geo_landings.reclat != 0)]

plot_single_year.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
plt.title("Meteor Falls in 2012", size = 20)
ax.axis('off')
plt.show()

Nous y allons! Cela a pris soin de cette vue d'atterrissage particulière (loin) au large des côtes de l'Afrique. Maintenant que nous pouvons créer des cartes avec des données sur une seule année, mettons tout cela dans un widget interactif afin que nous puissions consulter n'importe quelle année.

## Le mettre dans un widget

Dans la cellule ci-dessous, nous créons le widget que nous pouvons utiliser pour visualiser chaque année que nous aimons. Le code ici semble intimidant, mais il y a environ 95% de commentaires expliquant ce qui se passe. Le code réel du widget n'est qu'une somme dérisoire à environ 12 lignes de code!

In [ ]:
'''
This function is actually nothing new! Instead of plotting our graph directly, we've hidden it 
in a function to call instead. We've also called it 'update' as it is the function that will update
our plot with new data. In this case, our function takes a single input 'y' for year, this is the input 
that is used to filter our data down to a single year. But, everything in the function below is exactly
the plots we've already used. 
'''
def update(y):
    ax = world.plot(figsize=(20,10), linewidth=0.25, edgecolor='white', color='lightgrey')
    
    # Notice how we're specifying the year exactly as we did before, now we're just using
    # the variable 'y' to select the year instead!
    geo_landings[(geo_landings.year.dt.year == y) & (geo_landings.reclat !=0)].plot(alpha=0.75, 
                                                                                    ax=ax,
                                                                                    column = "fall",
                                                                                    legend = True)
    plt.title("Meteorite Falls in " + str(int(y)), size = 20)
    ax.axis('off')
    plt.show()

'''
Here we're simply finding and sorting all the years which had meteorites fall. Here's a quick overview
of what each funciton is doing below, as there's actually a lot to digest in that short line. 

sorted() : This function sorts data in ascending order (smallest -> middlest -> biggest) 
           If it is not simply numerical data, it will be sorted alphanumerically. 
           
list()   : This function simply turns what is inside parenthesis into a python list

landings.year.dt.year.unique() : We've actually seen this before to extract just the year. However this
                                 time we've also added the .unique() method. This returns a filtered list 
                                 of only the unique years that meteorites were observed to fall or found. 
                                   
[:-1]  : This is a short hand to parse our list. In this case the colon specifies that we're taking 
         elements from the beginning of the list. The -1 like this specifies that we're taking the 
         whole list, with the exception of the final element. This is because the year in the final 
         element has an error, and our widget won't work at that point. 
'''

years_with_fall = sorted(list(landings.year.dt.year.unique()))[:-1]


'''
The contents of this code is explained below.

interact_manual             : This is the function that we imported earlier that allows us to create an interactive
                              widget taht will help us explore the data. The _manual suffix tells us that we also
                              want to create a button to only update our plot when we've found the year we desire. 
                  
update                      : This is the function we defined at the beginning of the cell, and the function that 
                              interact will continuously update for us. 

y = widgets.SelectionSlider : This is the widget that we will be using to pass values to our 'update' function. 
                              Notice how we have used the name 'y', the same argument that our update() parameter
                              takes. In this case, SelectionSlider specifies that we want to use a slider widget
                              in order to select the values of y, the year that we're interested in. The arguments
                              of Selection Slider are explaned below. 
                              
description                 : This is a string (computer science talk for "a bunch of text") that will be the label
                              for our slider.
                            
options                     : This is a list of values our slider can take. In ourcase, it is the list we created 
                              earlier where we know meteors have fallen. 
'''

interact_manual(update, 
         y=widgets.SelectionSlider(description='Select Year', 
                                   options=years_with_fall))

Nous notons que vous devrez cliquer sur "Run Interact" après avoir ajusté votre curseur à l'année de votre choix. Nous avons démontré à quel point il est facile de créer un widget interactif à l'aide de Jupyter. Si vous voulez en savoir plus, la documentation des widgets est disponible [sur ce lien](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html). Notez que ce lien va au didacticiel de base de la documentation plutôt qu'à la page d'accueil.

# Conclusion

Ce bloc-notes a montré comment utiliser des ensembles de données ouverts contenant un composant géo-spatial. Plus précisément, nous vous avons montré comment créer des cartes en utilisant ces données. Nous avons également démontré comment vous pourriez ajouter une petite partie de l'interactivité pour voir la chute de météorites au cours d'une année donnée. Dans le prochain tutoriel, nous approfondirons l'analyse des données et verrons si nous pouvons découvrir quelque chose d'intéressant caché dans les données.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)